In [1]:
import weave
import typing

In [ ]:
ItemType = typing.TypeVar('ItemType')

@weave.type()
class Distribution(weave.panels.Panel):
    input_node: weave.Node[list[typing.Any]]
    series_fn: typing.Callable[[ItemType], list[float]] = lambda x: x
    series_label: typing.Callable[[ItemType], str] = None
    n_bins: int = 10 # TODO: generalize to other binning methods
        
    # Configuration
    #   series expression: expression from incoming Item to a series
    #   series label: expression from incoming item to a label    
    
    def render_config():
        # TODO: how
        pass
    
    def render():
        bin_fn = weave.ops.equal_bins(.1)
        return weave.panels.Plot(
            self.input_node,
            
            label=self.series_label,
            group_by_label=True,
            
            x=lambda row, domain: self.series_expression(row).bin(
                weave.ops.equal_bins(domain.min(), domain.max(), self.n_bins)
            ),
            group_by_x=True,
            
            y=lambda bin_values: bin_values.count()
        )

In [ ]:
# Simple histogram over input series
Distribution([random.random() for i in range(1000)])

In [ ]:
runs = [
    Run(name='a', history=[{'a': random.random() for i in range(1000)}]),
    Run(name='b', history=[{'a': random.random() for i in range(1000)}])
]

# Label automatically inferred because item is Run
Distribution(runs, series_fn=lambda run: run.history['a'])

In [ ]:
# Add KL divergence
weave.panels.Group(
    items={
        'distribution_plot': Distribution(runs, series_fn=lambda run: run.history['a']),
        'summary': lambda distribution_plot:
                weave.ops.kldivergence(
                    distribution_plot.series[0],
                    distribution_plot.series[1])
    }
)

In [2]:
# 2nd implementation
ItemType = typing.TypeVar('ItemType')

@weave.type()
class Distribution(weave.panels.Panel):
    input_node: weave.Node[list[typing.Any]]
    series_fn: typing.Callable[[ItemType], list[float]] = lambda x: x
    series_label: typing.Callable[[ItemType], str] = None
    n_bins: int = 10 # TODO: generalize to other binning methods
        
    @weave.op()
    def series():
        return self.input_node.map(
            lambda r: {'label': self.series_label(r), 'values': self.series_fn(r)}
        )
    
    @weave.op()
    def bin_fn():
        all_values = self.series().unnest()['values']
        return weave.ops.equal_bins(all_values.min(), all_values.max(), self.n_bins)
    
    @weave.op()
    def distributions():
        return self.series.map(
            lambda series: {
                'label': series['label'],
                'value_counts': series['values']
                    .group_by(lambda v: v.bin(self.bin_fn()))
                    .count()
            }
        )
    
    def render():
        return weave.panels.Plot(
            self.distributions().unnest(),
            label=lambda v: v['label'],
            x=lambda v: row['value'][0]
            y=lambda v: row['value'][1]
        )

SyntaxError: invalid syntax (2839749518.py, line 38)

In [ ]:
import weave
weave.use_frontend_devmode()
import random
data = weave.save([random.random() for i in range(100)])
data